In [ ]:
import json
import joblib
import pandas as pd
import numpy as np
from datetime import datetime

# --- Load Models and Data ---
try:
    print("Loading models and historical data...")
    model_temp = joblib.load('regional_model_temperature.joblib')
    model_rain = joblib.load('regional_model_rain.joblib')
    model_wind_u = joblib.load('regional_model_wind_u.joblib')
    model_wind_v = joblib.load('regional_model_wind_v.joblib')
    model_humidity = joblib.load('regional_model_humidity.joblib')
    historical_means = pd.read_pickle('historical_means_regional.pkl')
    print("✅ Models loaded successfully.")
except FileNotFoundError as e:
    print(f"❌ Critical error: File not found '{e.filename}'.")
    print("Make sure all .joblib and .pkl files are in the same folder as this script.")
    exit()

# --- Helper Functions ---
def get_closest_historical_data(lat, lon, doy, hour):
    """
    Finds the closest historical data manually.
    """
    try:
        subset = historical_means.loc[(doy, hour)]
        if subset.empty:
            raise KeyError
        subset = subset.reset_index()
        distances = (subset['lat'] - lat)**2 + (subset['lon'] - lon)**2
        closest_index = distances.idxmin()
        return subset.iloc[closest_index]
    except KeyError:
        print(f"WARNING: No data found for day {doy}, hour {hour}. Searching for the closest point in the entire history.")
        temp_df = historical_means.reset_index()
        distances = (temp_df['lat'] - lat)**2 + (temp_df['lon'] - lon)**2 + (temp_df['dayofyear'] - doy)**2
        closest_index = distances.idxmin()
        return temp_df.iloc[closest_index]

def convert_wind_to_speed_dir(u, v):
    """Converts U/V wind components to speed and cardinal direction."""
    speed_mps = np.sqrt(u**2 + v**2)
    speed_kmh = speed_mps * 3.6
    direction_degrees = (270 - np.rad2deg(np.arctan2(v, u))) % 360
    dirs = ["North", "Northeast", "East", "Southeast", "South", "Southwest", "West", "Northwest"]
    ix = int(round(direction_degrees / 45))
    cardinal_direction = dirs[ix % 8]
    return speed_kmh, direction_degrees, cardinal_direction

# --- Main Prediction Function ---
def get_forecast(lat, lon, timestamp_str):
    """
    Performs a weather prediction for a specific location and time.
    """
    try:
        date_obj = datetime.fromisoformat(timestamp_str)
        dayofyear = date_obj.timetuple().tm_yday
        hour = date_obj.hour
        
        original_features = ["TLML", "PRECTOTCORR", "QLML", "ULML", "VLML"]
        
        base_features = get_closest_historical_data(lat, lon, dayofyear, hour)
        X_pred_full = pd.DataFrame([base_features])
        X_pred = X_pred_full[original_features]
        
        temp_pred = model_temp.predict(X_pred.drop('TLML', axis=1))
        rain_prob_pred = model_rain.predict_proba(X_pred.drop('PRECTOTCORR', axis=1))
        wind_u_pred = model_wind_u.predict(X_pred.drop('ULML', axis=1))
        wind_v_pred = model_wind_v.predict(X_pred.drop('VLML', axis=1))

        if rain_prob_pred.shape[1] > 1:
            rain_probability = rain_prob_pred[0, 1]
        else:
            if model_rain.classes_[0] == 0:
                rain_probability = 0.0
            else:
                rain_probability = 1.0
        
        wind_speed_kmh, direction_degrees, cardinal_direction = convert_wind_to_speed_dir(wind_u_pred[0], wind_v_pred[0])

        return {
            "success": True, "date_str": date_obj.strftime('%Y-%m-%d %H:%M:%S'),
            "lat": lat, "lon": lon, "temp_pred_kelvin": temp_pred, "rain_probability": rain_probability,
            "wind_speed_kmh": wind_speed_kmh, "direction_degrees": direction_degrees,
            "cardinal_direction": cardinal_direction
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

# --- Main Execution Block ---
if __name__ == "__main__":
    
    # --- MODIFY THESE VALUES FOR YOUR TESTS! ---
    TEST_LATITUDE = -16.4090
    TEST_LONGITUDE = -71.5375
    TEST_DATE = "2024-01-04T12:00:00"
    # -------------------------------------------
    
    print(f"\n🚀 Running forecast for Lat/Lon: {TEST_LATITUDE}, {TEST_LONGITUDE}")
    result = get_forecast(TEST_LATITUDE, TEST_LONGITUDE, TEST_DATE)
    
    if result["success"]:
        # Temperature conversion
        temp_kelvin = result['temp_pred_kelvin'][0]
        temp_celsius = temp_kelvin - 273.15
        temp_fahrenheit = (temp_celsius * 9/5) + 32
        
        # Formatted results printing
        print("\n" + "="*65)
        print(f"--- Forecast for {result['date_str']} ---")
        print(f"📍 Location: Lat {result['lat']:.2f}, Lon {result['lon']:.2f}")
        print(f"🌡️ Expected Temperature: {temp_celsius:.1f}°C  /  {temp_fahrenheit:.1f}°F")
        print(f"💧 Probability of Rain: {result['rain_probability']:.0%}")
        print(f"💨 Wind: {result['wind_speed_kmh']:.1f} km/h from the {result['cardinal_direction']} direction ({result['direction_degrees']:.0f}°)")
        print("="*65)
        
    else:
        print("\n❌ An error occurred during the prediction:")
        print(result["error"])

Loading models and historical data...
✅ Models loaded successfully.

🚀 Running forecast for Lat/Lon: -16.409, -71.5375

--- Forecast for 2024-01-04 12:00:00 ---
📍 Location: Lat -16.41, Lon -71.54
🌡️ Expected Temperature: 8.5°C  /  47.4°F
💧 Probability of Rain: 0%
💨 Wind: 4.8 km/h from the Northwest direction (335°)
